<a href="https://colab.research.google.com/github/K-107/BERTsDay/blob/main/BertTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 사전 작업

In [ ]:
# GPU 정보 보기
! nvidia-smi --query | fgrep 'Product Name'

    Product Name                          : Tesla P4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 리포지토리 복제
%cd /content/
! git clone https://github.com/K-107/BERTsDay.git

/content
Cloning into 'BERTsDay'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 142 (delta 54), reused 66 (delta 16), pack-reused 0
Receiving objects: 100% (142/142), 63.93 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
# 밑의 모듈은 텐서플로 1.x로 작성되었으므로 버전 변경
! pip install --upgrade tensorflow==1.15.0rc3

     |████████████████████████████████| 412.3MB 36kB/s 
     |████████████████████████████████| 512kB 49.3MB/s 
     |████████████████████████████████| 3.8MB 51.3MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=07437fd427f2c2c86e38c43f02a67c52141d29ccf84421fa9a75bc04b7a58d16
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

## pretrained BERT 모델을 모듈로 export

ETRI에서 사전훈련한 BERT의 체크포인트를 가지고 BERT 모듈을 만드는 과정.

In [ ]:
! if [ ! -d /content/drive/MyDrive/bert_slottagging/ ]; then echo "프리트레인 모듈화 진행중"; python /content/BERTsDay/Bert_fine_tuning/export_korbert/bert_to_module.py -i "/content/drive/MyDrive/bert_slottagging/004_bert_eojeol_tensorflow" -o "/content/drive/MyDrive/bert_slottagging/Bert_pretrained/"; echo"완료"; else echo "프리트레인 모듈화 이미 완료됨"; fi

프리트레인 모듈화 이미 완료됨


## 데이터 준비

모델을 훈련하기 위해 필요한 seq.in, seq.out이라는 2가지 파일을 만드는 과정.

**(중요) 사전에 프로젝트 폴더 아래에 sample이란 폴더가 있어야 하며, Full_data.txt 또는 sample_input.txt 파일이 있어야 합니다.**


In [ ]:
%cd /content/drive/MyDrive/bert_slottagging
! if [ -d ./sample ]; then echo "sample 폴더 존재"; else echo "sample 폴더가 만들어졌습니다, sample 폴더에 sample_input.txt파일을 넣어주세요"; mkdir "./sample"; fi
%cd /content/drive/MyDrive/bert_slottagging/sample

/content/drive/MyDrive/bert_slottagging
sample 폴더 존재
/content/drive/MyDrive/bert_slottagging/sample


In [ ]:
# 전체 데이터에서 일부 문장 랜덤 추출 (선택사항)
! shuf -n 1000000 "Full_data.txt" > "sample_input.txt"

In [ ]:
#1. 텍스트 파일의 문장들을 무작위로 뒤섞는 명령어
! shuf "sample_input.txt" > "temp_0.txt"
# 2. 공백 문자 여러개를 하나로 줄이는 명령어
! sed 's/  */ /g' "temp_0.txt" > "temp_1.txt"
#3. 앞, 뒤의 공백 문자를 없애는 명렁어
#앞 :
! sed 's/^ //g' "temp_1.txt" > "temp_2.txt"
#뒤 :
! sed 's/ $//g' "temp_2.txt" > "temp_3.txt"
#4. 중복 제거하는 명령어
! sort -u "temp_3.txt" > "sample_output.txt"
# 임시 파일들 제거
! rm "temp_0.txt"
! rm "temp_1.txt"
! rm "temp_2.txt"
! rm "temp_3.txt"

In [ ]:
# 문장 갯수 세고, 파일 쪼개기
sentence_count = !wc -l "sample_output.txt" | grep -o "[0-9]*"
sentence_count = int(sentence_count[0])
train_count = int(sentence_count*0.6)
test_count = int(sentence_count*0.2)
val_count = sentence_count - train_count - test_count

!echo "전체 문장 개수: $sentence_count, 훈련: $train_count, 테스트: $test_count, 검증: $val_count"
!split -l $train_count "sample_output.txt" "sample_"
!mv "sample_aa" "train.txt"
!head -n $val_count "sample_ab" > "val.txt"
!tail -n $test_count "sample_ab" > "test.txt"
!rm "sample_ab"
!rm "sample_input.txt"

전체 문장 개수: 99992, 훈련: 59995, 테스트: 19998, 검증: 19999


In [ ]:
# 쪼갠 파일을 각각의 폴더에 seq.in, seq.out으로 저장
! for file in "train.txt" "test.txt" "val.txt"; do if [ -f $file ]; then folder=`echo $file | cut -f 1 -d '.'`; if [ ! -d "$folder" ]; then mkdir "$folder"; echo "$folder 폴더 생성"; fi; echo "$folder 데이터 준비중"; python /content/BERTsDay/Bert_fine_tuning/prepare_data.py -i "$file" -o "$PWD/$folder" -vp "/content/drive/MyDrive/bert_slottagging/Bert_pretrained/assets/vocab.korean.rawtext.list"; echo "완료"; else echo "파일 $file 없음"; fi; done

train 데이터 준비중

완료
test 데이터 준비중

완료
val 데이터 준비중

완료


### Fine-tuing 훈련

어떻게 하면 train_bert_finetuning.py 코드를 실행할 수 있는지 코드 내부의 parser을 참조하여 작성하세요.

In [ ]:
# 가용한 VRAM에 따라서 -bs (배치 사이즈)를 2^n로 유연하게 조절해서 쓰시면 됩니다.
%cd /content/drive/MyDrive/bert_slottagging/
! python /content/BERTsDay/Bert_fine_tuning/train_bert_finetuning.py -bp "./Bert_pretrained/" -t "./sample/train/" -v "./sample/val/" -s "./Fine_tuned/" -e 3 -bs 64 -tp bert

/content/drive/MyDrive/bert_slottagging

2021-04-15 07:47:54.525540: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-15 07:47:54.530366: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-04-15 07:47:54.530562: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5573c1f0c680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-15 07:47:54.530591: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-15 07:47:54.534198: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-15 07:47:54.652713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NU

## 모델 평가

테스트의 결과는 --model에 넣어준 모델 경로 아래의 test_results에 저장된다.

In [ ]:
%cd /content/drive/MyDrive/bert_slottagging/
! python /content/BERTsDay/Bert_fine_tuning/eval_bert_finetuned.py -bp "./Bert_pretrained/" -m "./Fine_tuned/" -d "./sample/test/" -tp bert

/content/drive/MyDrive/bert_slottagging


2021-04-15 08:41:53.822788: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-15 08:41:53.828078: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-04-15 08:41:53.828522: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5560079f6680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-15 08:41:53.828556: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-15 08:41:53.830494: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-15 08:41:53.835946: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-15 08:41:53.836001: I tensorflow/str

## Inference (임의의 문장을 모델에 넣어보기)

모델 자체가 용량이 커서 불러오는 데까지 시간이 걸림  
"Enter your sentence:"라는 문구가 나오면 모델에 넣어보고 싶은 문장을 넣어 주면 됨  
quit라는 입력을 넣어 주면 종료  

In [ ]:
%cd /content/drive/MyDrive/bert_slottagging/
!python /content/BERTsDay/Bert_fine_tuning/inference.py -bp "./Bert_pretrained/" -m "./Fine_tuned/" -tp bert

/content/drive/MyDrive/bert_slottagging

2021-04-15 08:37:58.552781: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-15 08:37:58.557578: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-04-15 08:37:58.557725: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561b1b15a680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-15 08:37:58.557752: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-15 08:37:58.559358: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-15 08:37:58.565438: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-15 08:37:58.565476: I tensorflow/stre